In [1]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.metrics import TopKCategoricalAccuracy
import glob

In [3]:
cfg = tf.compat.v1.ConfigProto() 
cfg.gpu_options.per_process_gpu_memory_fraction = 0.8 # 使用50%的GPU暫存  
session = tf.compat.v1.Session(config=cfg )

## 把預測結果集成

In [5]:
submission_template = pd.read_csv('result/1128/sail_1128_reslut1.csv',header=None)

In [6]:
chars = 'abcdefghijklmnopqrs'
chartonumbers = {k:v for k,v in enumerate(chars)}

def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

In [7]:
my_list = list(range(361))
reshaped_array = np.reshape(np.array(my_list), (19, 19))
reshaped_array

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18],
       [ 19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
         32,  33,  34,  35,  36,  37],
       [ 38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
         51,  52,  53,  54,  55,  56],
       [ 57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75],
       [ 76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
         89,  90,  91,  92,  93,  94],
       [ 95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107,
        108, 109, 110, 111, 112, 113],
       [114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132],
       [133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145,
        146, 147, 148, 149, 150, 151],
       [152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
        165, 166, 167, 168, 169

### public dan

In [9]:
#last1
npz_path = 'E:/research/AIcup_GO/PrivateData/weight/Public_weight_NPZ/Dan/'
npz_path_list =[
    # npz_path +'',
    npz_path +'training_Dan_1125_newf10_batch300000_incep_attention(concat)_loss.npz',
    npz_path +'training_Dan_1123_newf18_batch100000_uptodown_top1.npz']

npz_path_list

['E:/research/AIcup_GO/PrivateData/weight/Public_weight_NPZ/Dan/training_Dan_1125_newf10_batch300000_incep_attention(concat)_loss.npz',
 'E:/research/AIcup_GO/PrivateData/weight/Public_weight_NPZ/Dan/training_Dan_1123_newf18_batch100000_uptodown_top1.npz']

In [10]:
all_add = np.zeros((11000,19,19), dtype=float)
for path in npz_path_list:
    loaded_data = np.load(path)
    predict = loaded_data['predictions']
    
    all_add  += predict

all_add.shape      

(11000, 19, 19)

In [11]:
all_add = all_add.reshape((-1,361))
all_add.shape  

(11000, 361)

In [12]:
# 對於每一個預測結果，找出分數最高的 top1 和 top5 的索引
top1_indices = np.argmax(all_add, axis=-1)
top5_indices = np.argsort(all_add, axis=-1)[:, -5:]

# 輸出 top1 和 top5 的預測結果
print("Top1 Predicted Indices: ", top1_indices)
print("Top5 Predicted Indices: ", top5_indices)

Top1 Predicted Indices:  [220 239 104 ... 178 140 236]
Top5 Predicted Indices:  [[256 219 221 237 220]
 [107 220 294 275 239]
 [ 70  67 218  68 104]
 ...
 [335 255 334  30 178]
 [173 141 158 121 140]
 [214 233 237 196 236]]


In [13]:
chars = 'abcdefghijklmnopqrs'
answer = np.full((11000, 5), "", dtype=object)
for i  in range(len(top1_indices)):
    for j in range(1,6):
        num = top5_indices[i][j*(-1)] 
        indices = np.where(reshaped_array == num)
        answer[i][j-1] = chars[int(indices[1])]  + chars[int(indices[0])] 
        

In [14]:
answer = answer.astype('<U2')
answer

array([['ll', 'jm', 'ml', 'kl', 'jn'],
       ['lm', 'jo', 'jp', 'll', 'mf'],
       ['jf', 'ld', 'jl', 'kd', 'nd'],
       ...,
       ['hj', 'lb', 'lr', 'in', 'mr'],
       ['hh', 'hg', 'gi', 'ih', 'cj'],
       ['im', 'gk', 'jm', 'fm', 'fl']], dtype='<U2')

In [15]:
submission_template.iloc[11339][0]

'KTPU0000011340'

In [16]:
submission_template.iloc[11338:11350]

,0,1,2,3,4,5
11338,KTPU0000011339,aa,bb,cc,dd,ee
11339,KTPU0000011340,aa,bb,cc,dd,ee
11340,DTPU0000000001,aa,bb,cc,dd,ee
11341,DTPU0000000002,aa,bb,cc,dd,ee
11342,DTPU0000000003,aa,bb,cc,dd,ee
11343,DTPU0000000004,aa,bb,cc,dd,ee
11344,DTPU0000000005,aa,bb,cc,dd,ee
11345,DTPU0000000006,aa,bb,cc,dd,ee
11346,DTPU0000000007,aa,bb,cc,dd,ee
11347,DTPU0000000008,aa,bb,cc,dd,ee


In [17]:
for i in range(len(answer)):
    for a in range(0,5):
        submission_template.iloc[11340 + i][a+1] = answer[i][a]

In [18]:
submission_template.iloc[11338:11350]

,0,1,2,3,4,5
11338,KTPU0000011339,aa,bb,cc,dd,ee
11339,KTPU0000011340,aa,bb,cc,dd,ee
11340,DTPU0000000001,ll,jm,ml,kl,jn
11341,DTPU0000000002,lm,jo,jp,ll,mf
11342,DTPU0000000003,jf,ld,jl,kd,nd
11343,DTPU0000000004,no,bd,qm,rk,sj
11344,DTPU0000000005,nc,ec,fq,nd,ed
11345,DTPU0000000006,ke,kf,je,kd,le
11346,DTPU0000000007,lq,mq,hq,fq,gq
11347,DTPU0000000008,hq,jq,kp,lp,hr


In [19]:
submission_template.iloc[22338:22350]

,0,1,2,3,4,5
22338,DTPU0000010999,hh,hg,gi,ih,cj
22339,DTPU0000011000,im,gk,jm,fm,fl
22340,PSTPU0000000001_99,1,NaN,NaN,NaN,NaN
22341,PSTPU0000000001_153,1,NaN,NaN,NaN,NaN
22342,PSTPU0000000001_131,1,NaN,NaN,NaN,NaN
22343,PSTPU0000000001_21,2,NaN,NaN,NaN,NaN
22344,PSTPU0000000001_79,1,NaN,NaN,NaN,NaN
22345,PSTPU0000000001_7,1,NaN,NaN,NaN,NaN
22346,PSTPU0000000001_19,2,NaN,NaN,NaN,NaN
22347,PSTPU0000000001_55,3,NaN,NaN,NaN,NaN


### public kyu

In [21]:
#last1
npz_path = 'E:/research/AIcup_GO/PrivateData/weight/Public_weight_NPZ/Kyu/'
npz_path_list =[
    # npz_path +'',
    npz_path +'training_Kyu_1127_newf10_batch400000_incep_atten_top1_2.npz',
    npz_path +'training_Kyu_1121_newf18_incep_attention(concat)_top1.npz']

npz_path_list

['E:/research/AIcup_GO/PrivateData/weight/Public_weight_NPZ/Kyu/training_Kyu_1127_newf10_batch400000_incep_atten_top1_2.npz',
 'E:/research/AIcup_GO/PrivateData/weight/Public_weight_NPZ/Kyu/training_Kyu_1121_newf18_incep_attention(concat)_top1.npz']

In [22]:
# npz_path = 'E:/research/AIcup_GO/PrivateData/weight/Public_weight_NPZ/Kyu/'
# # npz_path_list = glob.glob(npz_path)
# npz_path_list =[
# npz_path +'training_Dan_1010_inceptionblock2_loss.npz' ,
# npz_path + 'training_Kyu_1121_newf18_incep_attention(concat)_top1.npz',
# npz_path +'training_Kyu_1121_newf18_incep_uptodown_top1.npz',
# npz_path+ 'training_Dan_1125_newf10_batch300000_incep_attention(concat)_top1.npz']
# # npz_path_list.pop(2)
# npz_path_list

In [23]:
all_add = np.zeros((11340,19,19), dtype=float)
for path in npz_path_list:
    loaded_data = np.load(path)
    predict = loaded_data['predictions']
    print(predict.shape)
    all_add  += predict

all_add.shape      

(11340, 19, 19)
(11340, 19, 19)


(11340, 19, 19)

In [24]:
all_add = all_add.reshape((-1,361))
all_add.shape  

(11340, 361)

In [25]:
# 對於每一個預測結果，找出分數最高的 top1 和 top5 的索引
top1_indices = np.argmax(all_add, axis=-1)
top5_indices = np.argsort(all_add, axis=-1)[:, -5:]

# 輸出 top1 和 top5 的預測結果
print("Top1 Predicted Indices: ", top1_indices)
print("Top5 Predicted Indices: ", top5_indices)

Top1 Predicted Indices:  [ 13 284 231 ... 212 183  39]
Top5 Predicted Indices:  [[ 14  11  33  32  13]
 [307 226 354 242 284]
 [249 230 325 268 231]
 ...
 [ 97 311 191 210 212]
 [189 239 208  11 183]
 [ 64  59  46  45  39]]


In [26]:
answer = np.full((11340, 5), "", dtype=object)
for i  in range(len(top1_indices)):
    for j in range(1,6):
        num = top5_indices[i][j*(-1)] 
        indices = np.where(reshaped_array == num)
        answer[i][j-1] = chars[int(indices[1])]  + chars[int(indices[0])] 
        

In [27]:
answer = answer.astype('<U2')
answer

array([['na', 'nb', 'ob', 'la', 'oa'],
       ['so', 'om', 'ms', 'rl', 'dq'],
       ['dm', 'co', 'cr', 'cm', 'cn'],
       ...,
       ['dl', 'bl', 'bk', 'hq', 'cf'],
       ['mj', 'la', 'sk', 'lm', 'sj'],
       ['bc', 'hc', 'ic', 'cd', 'hd']], dtype='<U2')

In [28]:
submission_template.head()

,0,1,2,3,4,5
0,KTPU0000000001,aa,bb,cc,dd,ee
1,KTPU0000000002,aa,bb,cc,dd,ee
2,KTPU0000000003,aa,bb,cc,dd,ee
3,KTPU0000000004,aa,bb,cc,dd,ee
4,KTPU0000000005,aa,bb,cc,dd,ee


In [29]:
for i in range(len(answer)):
    for a in range(0,5):
        submission_template.iloc[i][a+1] = answer[i][a]

In [30]:
submission_template.head()

,0,1,2,3,4,5
0,KTPU0000000001,na,nb,ob,la,oa
1,KTPU0000000002,so,om,ms,rl,dq
2,KTPU0000000003,dm,co,cr,cm,cn
3,KTPU0000000004,fr,gs,ck,fk,fj
4,KTPU0000000005,jc,gb,jb,kb,gc


### private dan

In [32]:
#last1
npz_path = 'E:/research/AIcup_GO/PrivateData/weight/Dan/predict_NPZ/'
npz_path_list =[
    # npz_path +'',
    npz_path +'training_Dan_1125_newf10_batch300000_incep_attention(concat)_loss.npz',
    npz_path +'training_Dan_1123_newf18_batch100000_uptodown_top1.npz']

npz_path_list

['E:/research/AIcup_GO/PrivateData/weight/Dan/predict_NPZ/training_Dan_1125_newf10_batch300000_incep_attention(concat)_loss.npz',
 'E:/research/AIcup_GO/PrivateData/weight/Dan/predict_NPZ/training_Dan_1123_newf18_batch100000_uptodown_top1.npz']

In [33]:
all_add = np.zeros((11000,19,19), dtype=float)
for path in npz_path_list:
    loaded_data = np.load(path)
    predict = loaded_data['predictions']
    print(predict.shape)
    all_add  += predict

all_add.shape      

(11000, 19, 19)
(11000, 19, 19)


(11000, 19, 19)

In [34]:
all_add = all_add.reshape((-1,361))
all_add.shape  

(11000, 361)

In [35]:
# 對於每一個預測結果，找出分數最高的 top1 和 top5 的索引
top1_indices = np.argmax(all_add, axis=-1)
top5_indices = np.argsort(all_add, axis=-1)[:, -5:]

# 輸出 top1 和 top5 的預測結果
print("Top1 Predicted Indices: ", top1_indices)
print("Top5 Predicted Indices: ", top5_indices)

Top1 Predicted Indices:  [258 190 202 ...  55 154 183]
Top5 Predicted Indices:  [[221 359 292 278 258]
 [209 152 210 191 190]
 [140 184 124 121 202]
 ...
 [309 168 109  73  55]
 [305 155 286 330 154]
 [141 160 165 184 183]]


In [36]:
answer = np.full((11000, 5), "", dtype=object)
for i  in range(len(top1_indices)):
    for j in range(1,6):
        num = top5_indices[i][j*(-1)] 
        indices = np.where(reshaped_array == num)
        answer[i][j-1] = chars[int(indices[1])]  + chars[int(indices[0])] 
        

In [37]:
answer = answer.astype('<U2')
answer

array([['ln', 'mo', 'hp', 'rs', 'ml'],
       ['ak', 'bk', 'bl', 'ai', 'al'],
       ['mk', 'hg', 'kg', 'nj', 'hh'],
       ...,
       ['rc', 'qd', 'of', 'qi', 'fq'],
       ['ci', 'hr', 'bp', 'di', 'bq'],
       ['mj', 'nj', 'ni', 'ii', 'ih']], dtype='<U2')

In [38]:
submission_template.iloc[43671:43671+10]

,0,1,2,3,4,5
43671,KTPR0000011336,aa,bb,cc,dd,ee
43672,KTPR0000011337,aa,bb,cc,dd,ee
43673,KTPR0000011338,aa,bb,cc,dd,ee
43674,KTPR0000011339,aa,bb,cc,dd,ee
43675,KTPR0000011340,aa,bb,cc,dd,ee
43676,DTPR0000000001,aa,bb,cc,dd,ee
43677,DTPR0000000002,aa,bb,cc,dd,ee
43678,DTPR0000000003,aa,bb,cc,dd,ee
43679,DTPR0000000004,aa,bb,cc,dd,ee
43680,DTPR0000000005,aa,bb,cc,dd,ee


In [39]:
for i in range(len(answer)):
    for a in range(0,5):
        submission_template.iloc[43676 + i][a+1] = answer[i][a]

In [40]:
submission_template.iloc[43671:43671+10]

,0,1,2,3,4,5
43671,KTPR0000011336,aa,bb,cc,dd,ee
43672,KTPR0000011337,aa,bb,cc,dd,ee
43673,KTPR0000011338,aa,bb,cc,dd,ee
43674,KTPR0000011339,aa,bb,cc,dd,ee
43675,KTPR0000011340,aa,bb,cc,dd,ee
43676,DTPR0000000001,ln,mo,hp,rs,ml
43677,DTPR0000000002,ak,bk,bl,ai,al
43678,DTPR0000000003,mk,hg,kg,nj,hh
43679,DTPR0000000004,ik,rh,nb,es,kh
43680,DTPR0000000005,sd,pi,pj,sh,pb


In [41]:
submission_template.iloc[54674:54674+10]

,0,1,2,3,4,5
54674,DTPR0000010999,ci,hr,bp,di,bq
54675,DTPR0000011000,mj,nj,ni,ii,ih
54676,PSTPR0000000001_127,1,NaN,NaN,NaN,NaN
54677,PSTPR0000000001_11,3,NaN,NaN,NaN,NaN
54678,PSTPR0000000001_79,2,NaN,NaN,NaN,NaN
54679,PSTPR0000000001_55,2,NaN,NaN,NaN,NaN
54680,PSTPR0000000001_43,1,NaN,NaN,NaN,NaN
54681,PSTPR0000000001_21,1,NaN,NaN,NaN,NaN
54682,PSTPR0000000001_39,2,NaN,NaN,NaN,NaN
54683,PSTPR0000000001_93,1,NaN,NaN,NaN,NaN


### private kyu

In [43]:
#last
npz_path = 'E:/research/AIcup_GO/PrivateData/weight/Kyu/predict_NPZ/'
npz_path_list =[
    # npz_path +'',
    npz_path +'training_Kyu_1127_newf10_batch400000_incep_atten_top1_2.npz',
    npz_path +'training_Kyu_1121_newf18_incep_attention(concat)_top1.npz']

npz_path_list

['E:/research/AIcup_GO/PrivateData/weight/Kyu/predict_NPZ/training_Kyu_1127_newf10_batch400000_incep_atten_top1_2.npz',
 'E:/research/AIcup_GO/PrivateData/weight/Kyu/predict_NPZ/training_Kyu_1121_newf18_incep_attention(concat)_top1.npz']

In [44]:
all_add = np.zeros((11340,19,19), dtype=float)
for path in npz_path_list:
    loaded_data = np.load(path)
    predict = loaded_data['predictions']
    print(predict.shape)
    all_add  += predict

all_add.shape      

(11340, 19, 19)
(11340, 19, 19)


(11340, 19, 19)

In [45]:
all_add = all_add.reshape((-1,361))
all_add.shape  

(11340, 361)

In [46]:
# 對於每一個預測結果，找出分數最高的 top1 和 top5 的索引
top1_indices = np.argmax(all_add, axis=-1)
top5_indices = np.argsort(all_add, axis=-1)[:, -5:]

# 輸出 top1 和 top5 的預測結果
print("Top1 Predicted Indices: ", top1_indices)
print("Top5 Predicted Indices: ", top5_indices)

Top1 Predicted Indices:  [ 39 109  23 ... 168 103 163]
Top5 Predicted Indices:  [[ 79 154 135 118  39]
 [ 89 148 279 128 109]
 [ 63  22  81  43  23]
 ...
 [283 150 166 169 168]
 [119 158 159 139 103]
 [ 48 140 220 201 163]]


In [47]:
answer = np.full((11340, 5), "", dtype=object)
for i  in range(len(top1_indices)):
    for j in range(1,6):
        num = top5_indices[i][j*(-1)] 
        indices = np.where(reshaped_array == num)
        answer[i][j-1] = chars[int(indices[1])]  + chars[int(indices[0])] 
        

In [48]:
answer = answer.astype('<U2')
answer

array([['bc', 'eg', 'ch', 'ci', 'de'],
       ['of', 'og', 'no', 'ph', 'ne'],
       ['eb', 'fc', 'fe', 'db', 'gd'],
       ...,
       ['qi', 'ri', 'oi', 'rh', 'ro'],
       ['if', 'gh', 'hi', 'gi', 'fg'],
       ['li', 'lk', 'll', 'hh', 'kc']], dtype='<U2')

In [49]:
submission_template[32333:32333 +10]

,0,1,2,3,4,5
32333,PSTPU0000000886_53,1,NaN,NaN,NaN,NaN
32334,PSTPU0000000886_11,3,NaN,NaN,NaN,NaN
32335,PSTPU0000000886_13,3,NaN,NaN,NaN,NaN
32336,KTPR0000000001,aa,bb,cc,dd,ee
32337,KTPR0000000002,aa,bb,cc,dd,ee
32338,KTPR0000000003,aa,bb,cc,dd,ee
32339,KTPR0000000004,aa,bb,cc,dd,ee
32340,KTPR0000000005,aa,bb,cc,dd,ee
32341,KTPR0000000006,aa,bb,cc,dd,ee
32342,KTPR0000000007,aa,bb,cc,dd,ee


In [50]:
for i in range(len(answer)):
    for a in range(0,5):
        submission_template.iloc[32336 +i][a+1] = answer[i][a]

In [51]:
submission_template[32333:32333 +10]

,0,1,2,3,4,5
32333,PSTPU0000000886_53,1,NaN,NaN,NaN,NaN
32334,PSTPU0000000886_11,3,NaN,NaN,NaN,NaN
32335,PSTPU0000000886_13,3,NaN,NaN,NaN,NaN
32336,KTPR0000000001,bc,eg,ch,ci,de
32337,KTPR0000000002,of,og,no,ph,ne
32338,KTPR0000000003,eb,fc,fe,db,gd
32339,KTPR0000000004,gl,im,gm,il,hk
32340,KTPR0000000005,fc,gc,ci,gb,bc
32341,KTPR0000000006,ap,bq,bp,aq,an
32342,KTPR0000000007,fc,gc,db,fb,ec


In [52]:
submission_template.to_csv('result/1128/_1128_reslut5.csv', index=False,header=None)